In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
#load modules
import numpy as np
from astropy import units as u
from astropy.io import fits

# import personal code
from model_kit import psd_functions as psd
from model_kit import datafiles as dfx

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#print('This notebook is not stale yet!')

In [2]:
%%time
opt_parms = { 'ovs': 4096,
             'surf_units': u.micron,
             'ca': 75,
             'ring_width': 3,
             'kmid_ll': 100/u.m, #0.1/u.mm,
             'khigh_ll': 1000/u.m} #1/u.mm}

parent_dir = '/home/jhen/XWCL/code/CACTI/'
data_folder = parent_dir + 'zygo_data/OAP_coated/'
oap_labels =  ('A', 'B', 'C', 'D', 'E', 'F')#, 'G', 'H')
oap=[]

for surface_name in oap_labels:
    n_step = 'OAP {0}'.format(surface_name)
    file_loc=data_folder+'oap{0}_{1}CA'.format(surface_name, opt_parms['ca'])
    # create the object
    opt = psd.surfPSD(surf_name = n_step)
    opt.open_surf(fileloc=file_loc+'_surf.fits', 
                  surf_units = opt_parms['surf_units'])
    opt.open_mask(fileloc=file_loc+'_mask.fits')
    opt.calc_psd(oversamp = opt_parms['ovs'], 
                 kmid_ll=opt_parms['kmid_ll'], 
                 khigh_ll=opt_parms['khigh_ll'])
    print('PSD for {0} complete'.format(n_step))
    oap.append(opt)
    # only calculating 2D PSD instead of radial because radial is slow and not needed for RMS

PSD for OAP A complete
PSD for OAP B complete
PSD for OAP C complete
PSD for OAP D complete
PSD for OAP E complete
PSD for OAP F complete
PSD for OAP AVG complete
CPU times: user 22.5 s, sys: 7.42 s, total: 29.9 s
Wall time: 18.8 s


### Calculate rms: beam diameter, DM correction
RMS values are for surface (on-axis) and OAP angle adjustment with reflection (basically, OPD).

In [3]:
def calc_strehl(rms_tot, wavelen):
    rms_tot = rms_tot.to(wavelen.unit)
    return np.exp(-(rms_tot.value*2*np.pi/wavelen.value)**2)

In [4]:
%%time
oap_angle = 23*u.deg # full angle
angle_scale = 2/np.cos(oap_angle/2) # go based on half angle
beam_diam = 0.0075*u.m # 7.5mm beam diamter
n_act_min = 12
n_act_max = 26
n_act_diam_list = np.linspace(n_act_min, n_act_max,
                              (n_act_max-n_act_min) + 1,
                              endpoint=True)

# step through each number of actuators across the surface
dm_rms = np.zeros((len(oap), len(n_act_diam_list)))
for j_oap in range(0, len(oap)):
    print('Evaluating {0}'.format(oap[j_oap].surf_name))
    kmax = oap[j_oap].k_max
    for j_act in range(0, len(n_act_diam_list)):
        bw_dm = n_act_diam_list[j_act]/(2*beam_diam)
        rms_val = psd.do_psd_rms(psd_data=oap[j_oap].psd_cal, 
                                 delta_k=oap[j_oap].delta_k, 
                                 k_tgt_lim=[bw_dm, kmax], 
                                 print_rms=False)
        # rms_val outputs as nanometers, but is removed for data storage
        dm_rms[j_oap][j_act] = rms_val.value * angle_scale # scaled for OAP angle and reflection

Evaluating OAP A
Evaluating OAP B
Evaluating OAP C
Evaluating OAP D
Evaluating OAP E
Evaluating OAP F
CPU times: user 50.7 s, sys: 18.4 s, total: 1min 9s
Wall time: 45.3 s


In [7]:
# calculate the strehls
wavelen = 633*u.nm
rms_bmc1k = 3*u.nm

# for dm_rms, the rows are every OAP
# the column is every DM actuator diamter count
# so, sum up according to each column.

tot_rms = np.zeros(len(n_act_diam_list))
sr = np.zeros_like(tot_rms)
for j in range(0, len(n_act_diam_list)):
    oap_rms = np.sum(dm_rms[:,j])
    tot_rms[j] = oap_rms + rms_bmc1k.value
    sr[j] = calc_strehl(tot_rms[j]*rms_bmc1k.unit, wavelen)
    print('N={0}, sigma={1:.3f}, SR = {2:.3f}'.format(n_act_diam_list[j], tot_rms[j], sr[j]))

N=12.0, sigma=28.340, SR = 0.924
N=13.0, sigma=26.916, SR = 0.931
N=14.0, sigma=25.735, SR = 0.937
N=15.0, sigma=24.302, SR = 0.943
N=16.0, sigma=23.206, SR = 0.948
N=17.0, sigma=22.397, SR = 0.952
N=18.0, sigma=21.604, SR = 0.955
N=19.0, sigma=20.904, SR = 0.958
N=20.0, sigma=20.199, SR = 0.961
N=21.0, sigma=19.533, SR = 0.963
N=22.0, sigma=18.956, SR = 0.965
N=23.0, sigma=18.384, SR = 0.967
N=24.0, sigma=17.835, SR = 0.969
N=25.0, sigma=17.315, SR = 0.971
N=26.0, sigma=16.801, SR = 0.973


Since 13 actuators produced 0.93 Strehl that matches Kyle's estimation, then go off that for all the rms values.

In [9]:
# now look at all the rms values
n_act_diam = 13 # solved previously
beam_diam = 0.0075*u.m # 7.5mm beam diameter
bw_beam = 1/beam_diam
bw_dm = n_act_diam/(2*beam_diam)

oap_angle = 23*u.deg # full angle
angle_scale = 2/np.cos(oap_angle/2) # go based on half angle

print('Cutoff Frequencies')
print('K_beam = {0:.3f}'.format(bw_beam))
print('K_DM = {0:.3f}'.format(bw_dm))
print('------')

tot_rms_beam = 0;
tot_rms_dm = 0;
for j in range(0, len(oap)):
    kmax = oap[j].k_max
    print('optic: {0}'.format(oap[j].surf_name))
    print('RMS (CA): {0:.3f}'.format(oap[j].rms_tot))
    #print('kmin={1:.4f}'.format(beam_diam[j], kmin))
    rms_val = psd.do_psd_rms(psd_data=oap[j].psd_cal, delta_k=oap[j].delta_k, 
                             k_tgt_lim=[bw_beam, kmax], 
                             print_rms=False)
    print('RMS (beam, on-axis): {0:.3f}'.format(rms_val))
    rms_beam_opd = rms_val * angle_scale
    print('RMS (beam, refl. angle): {0:.3f}'.format(rms_beam_opd))
    tot_rms_beam += rms_beam_opd
    # calculate DM rms
    rms_dm_val = psd.do_psd_rms(psd_data=oap[j].psd_cal, delta_k=oap[j].delta_k,
                               k_tgt_lim=[bw_dm, kmax], 
                               print_rms=False)
    print('RMS (DM, on-axis): {0:.3f}'.format(rms_dm_val))
    rms_dm_opd = rms_dm_val * angle_scale
    print('RMS (DM, refl. angle): {0:.3f}'.format(rms_dm_opd))
    tot_rms_dm += rms_dm_opd
    print('-----')

# Calculate the strehl
tot_rms_beam += rms_bmc1k
print('RMS tot (beam) = {0:.4f}'.format(tot_rms_beam))
sr_beam = calc_strehl(tot_rms_beam, wavelen)
print('SR (beam) = {0:.4f}'.format(sr_beam))
print('--')
tot_rms_dm += rms_bmc1k
print('RMS tot (DM) = {0:.4f}'.format(tot_rms_dm))
sr_dm = calc_strehl(tot_rms_dm, wavelen)
print('SR (DM) = {0:.4f}'.format(sr_dm))

Cutoff Frequencies
K_beam = 133.333 1 / m
K_DM = 866.667 1 / m
------
optic: OAP A
RMS (CA): 19.399 nm
RMS (beam, on-axis): 11.570 nm
RMS (beam, refl. angle): 23.614 nm
RMS (DM, on-axis): 1.552 nm
RMS (DM, refl. angle): 3.168 nm
-----
optic: OAP B
RMS (CA): 21.405 nm
RMS (beam, on-axis): 9.805 nm
RMS (beam, refl. angle): 20.012 nm
RMS (DM, on-axis): 1.970 nm
RMS (DM, refl. angle): 4.021 nm
-----
optic: OAP C
RMS (CA): 21.625 nm
RMS (beam, on-axis): 14.221 nm
RMS (beam, refl. angle): 29.025 nm
RMS (DM, on-axis): 1.586 nm
RMS (DM, refl. angle): 3.236 nm
-----
optic: OAP D
RMS (CA): 18.922 nm
RMS (beam, on-axis): 8.955 nm
RMS (beam, refl. angle): 18.277 nm
RMS (DM, on-axis): 1.799 nm
RMS (DM, refl. angle): 3.672 nm
-----
optic: OAP E
RMS (CA): 24.995 nm
RMS (beam, on-axis): 13.908 nm
RMS (beam, refl. angle): 28.386 nm
RMS (DM, on-axis): 2.659 nm
RMS (DM, refl. angle): 5.428 nm
-----
optic: OAP F
RMS (CA): 24.198 nm
RMS (beam, on-axis): 10.015 nm
RMS (beam, refl. angle): 20.441 nm
RMS (DM,

Out of pure curiousity, how does it look like with the average OAP?

In [10]:
# calculate the average PSD.
# The average PSD is calculated by averaging all the normalized PSDs, then mulitply by average variance.
avg_var = np.mean([obj.var.value for obj in oap])*oap[0].var.unit
avg_psd = np.zeros_like(oap[0].psd_cal.value)
for n in range(0, len(oap_labels)):
    avg_psd += oap[n].psd_norm.value
avg_psd /= len(oap_labels)
avg_psd = avg_psd * avg_var.value * oap[0].psd_cal.unit
print('PSD for OAP AVG complete')

PSD for OAP AVG complete


In [12]:
kmax = oap[0].k_max
print('optic: OAP AVG')
avg_rms = psd.do_psd_rms(psd_data=avg_psd, delta_k=oap[0].delta_k, 
                         k_tgt_lim=(oap[0].k_min, oap[0].k_max), print_rms=False)
print('RMS (CA): {0:.3f}'.format(avg_rms))
print('--')
avg_beam_rms = psd.do_psd_rms(psd_data=avg_psd, delta_k=oap[0].delta_k, 
                         k_tgt_lim=(bw_beam, oap[0].k_max), print_rms=False)
print('RMS (beam, on-axis): {0:.3f}'.format(avg_beam_rms))
avg_beam_rms_opd = avg_beam_rms * angle_scale
print('RMS (beam, refl. angle): {0:.3f}'.format(avg_beam_rms_opd))
# Calculate the strehl
avg_tot_rms_beam = (avg_beam_rms_opd*len(oap)) + rms_bmc1k
print('RMS tot (beam, refl. angle) = {0:.4f}'.format(avg_tot_rms_beam))
sr_beam = calc_strehl(avg_tot_rms_beam, wavelen)
print('SR (beam, refl. angle) = {0:.4f}'.format(sr_beam))
print('--')

avg_dm_rms = psd.do_psd_rms(psd_data=avg_psd, delta_k=oap[0].delta_k, 
                         k_tgt_lim=(bw_dm, oap[0].k_max), print_rms=False)
print('RMS (DM, on-axis): {0:.3f}'.format(avg_dm_rms))
avg_dm_rms_opd = avg_dm_rms * angle_scale
print('RMS (DM, refl. angle): {0:.3f}'.format(avg_dm_rms_opd))
# Calculate the strehl
avg_tot_rms_dm = (avg_dm_rms_opd*len(oap)) + rms_bmc1k
print('RMS tot (DM, refl. angle) = {0:.4f}'.format(avg_tot_rms_dm))
sr_dm = calc_strehl(avg_tot_rms_dm, wavelen)
print('SR (DM, refl. angle) = {0:.4f}'.format(sr_dm))

optic: OAP AVG
RMS (CA): 21.827 nm
--
RMS (beam, on-axis): 11.661 nm
RMS (beam, refl. angle): 23.801 nm
RMS tot (beam, refl. angle) = 145.8041 nm
SR (beam, refl. angle) = 0.1231
--
RMS (DM, on-axis): 1.960 nm
RMS (DM, refl. angle): 4.001 nm
RMS tot (DM, refl. angle) = 27.0066 nm
SR (DM, refl. angle) = 0.9307


Good to see that using the average PSD still matches closely with the original calculation.